In [1]:
%matplotlib inline

import torch
import numpy as np
import pandas as pd
from copy import deepcopy
from tabulate import tabulate
from torch import optim
import torchvision.utils
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import CrossEntropyLoss
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,random_split

import config
from utils import imshow
from models import SiameseNetwork, CNN
from training import trainSiamese,inferenceSiamese, trainCNN
from datasets import SiameseNetworkDataset, CNNDataset,generate_csv_compare
from loss_functions import ContrastiveLoss

# generate_csv(config.training_dir)

import os
if not os.path.exists('state_dict'):
    os.makedirs('state_dict')

## Generate Dateset with a few images per class

In [2]:
generate_csv_compare(config.training_dir,config.compare_siamese_csv,config.compare_cnn_csv,config.compare_test_csv,
                     num_per_class = 4)

Data directory:  ../../datasets/AT&T Database of Faces/faces/training


## Training Siamese Network

In [ ]:
# Split the dataset into train and validation sets
siamese_dataset = SiameseNetworkDataset(config.compare_siamese_csv,
                                        transform=transforms.Compose([
                                            transforms.Resize((config.img_height,config.img_width)),
                                            transforms.ToTensor(),
                                            transforms.Normalize(0,1)]),
                                        should_invert=False)

num_train = round(0.9*siamese_dataset.__len__())
num_validate = siamese_dataset.__len__()-num_train



siamese_train, siamese_valid = random_split(siamese_dataset, [num_train,num_validate])

siamese_train_dataloader = DataLoader(siamese_train,
                        shuffle=True,
                        num_workers=8,
                        batch_size=config.train_batch_size)

siamese_valid_dataloader = DataLoader(siamese_valid,
                        shuffle=True,
                        num_workers=8,
                        batch_size=1)

# Training
netS = SiameseNetwork().cuda()
criterionS = ContrastiveLoss()
optimizer = optim.Adam(netS.parameters(),lr = config.learning_rate )
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,config.step_size, config.gamma)

netS, train_loss_historyS, valid_loss_historyS,dict_nameS = trainSiamese(netS,criterionS,optimizer,scheduler,siamese_train_dataloader,
             siamese_valid_dataloader,config.train_number_epochs,do_show=True)

Epoch  0  training


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  2.08it/s]


Epoch  0  validating


100%|███████████████████████████████████████████| 21/21 [00:00<00:00, 57.31it/s]


+1/-1 ratrio:0.52/0.48
+1 features max:3.69	 min:0.78	 mean:2.02	 median 1.69
-1 features max:3.77	 min:1.97	 mean:3.04	 median 3.23
Feature = 0	Threshold = 2.286705	Polorization = 1
Epoch-0	 Train loss: 1.8813e+00	 Valid loss: 2.6000e+00	 Valid error: 0.1905
Epoch  1  training


...

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  3.66it/s]


Epoch  25  validating


100%|███████████████████████████████████████████| 21/21 [00:00<00:00, 59.15it/s]


+1/-1 ratrio:0.52/0.48
+1 features max:0.34	 min:0.08	 mean:0.21	 median 0.21
-1 features max:2.77	 min:0.84	 mean:1.54	 median 1.36
Feature = 0	Threshold = 0.591710	Polorization = 1
Epoch-25	 Train loss: 1.2210e-02	 Valid loss: 2.6768e-02	 Valid error: 0.0000
new model saved
Epoch  26  training


  0%|                                                     | 0/2 [00:00<?, ?it/s]

## Train CNN with Cross Entropy Loss

In [ ]:
cnn_dataset = CNNDataset(config.compare_cnn_csv,
                                        transform=transforms.Compose([
                                            transforms.Resize((config.img_height,config.img_width)),
                                            transforms.ToTensor(),
                                            transforms.Normalize(0,1)]),
                                        should_invert=False)

cnn_train_dataloader = DataLoader(cnn_dataset,
                        shuffle=True,
                        num_workers=8,
                        batch_size=config.train_batch_size)

netC = CNN().cuda()
criterionC = CrossEntropyLoss()
optimizer = optim.Adam(netC.parameters(),lr = config.learning_rate )
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,config.step_size, config.gamma)

In [ ]:
netC, train_loss_historyC, dict_nameC = trainCNN(netC,criterionC,optimizer,scheduler,cnn_train_dataloader,
             config.train_number_epochs,do_show=True)

In [ ]:
test_dataset = CNNDataset(config.compare_test_csv,
                                        transform=transforms.Compose([
                                            transforms.Resize((config.img_height,config.img_width)),
                                            transforms.ToTensor(),
                                            transforms.Normalize(0,1)]),
                                        should_invert=False)

test_dataloader = DataLoader(test_dataset,
                        shuffle=False,
                        num_workers=1,
                        batch_size=1)
true_labels = []
for _,label in iter(test_dataloader):
    true_labels.append(int(label))

netS = SiameseNetwork().cuda()
netS.load_state_dict(torch.load(os.path.join("state_dict",dict_nameS)))
netS.eval()

siamese_precalculate_dataloader = DataLoader(cnn_dataset,
                        shuffle=False,
                        num_workers=1,
                        batch_size=1)

feature_vectors = []
temp = []
current_label = 0 #int(next(iter(siamese_precalculate_dataloader))[1])
for data,label in iter(siamese_precalculate_dataloader):
    v = (netS.forward_once(data.cuda()).detach().cpu().numpy())[0]
    if label==current_label:
        temp.append(v)
    else:
        current_label = label
        feature_vectors.append(deepcopy(temp))
        temp = []
        temp.append(v)
feature_vectors.append(deepcopy(temp))

inferenceS = []
for data, label in iter(test_dataloader):
    current_feature = netS.forward_once(data.cuda())
    dissims = []
    temp = 0
    for vectors in feature_vectors:
        for feature in vectors:
            temp += F.pairwise_distance(current_feature,torch.tensor(feature).cuda()).detach().cpu().numpy()
        dissims.append(temp / len(vectors))
        temp = 0
    inferenceS.append(np.argmin(dissims))
    
netC = SiameseNetwork().cuda()
netC.load_state_dict(torch.load(os.path.join("state_dict",dict_nameC)))
netC.eval()

inferenceC = []
for data, label in iter(test_dataloader):
    current_feature = netC.forward_once(data.cuda())
    inferenceC.append(np.argmax(current_feature.detach().cpu().numpy()))

accS = sum([true_labels[i]==inferenceS[i] for i in range(len(true_labels))])/len(true_labels)
accC = sum([true_labels[i]==inferenceC[i] for i in range(len(true_labels))])/len(true_labels)

print(tabulate(np.array([true_labels,inferenceS,inferenceC]).T, headers=["true labels", "Siamese", "CNN"]))

print("Siamese accuracy: %.4f"%(accS))
print("Convnet accuracy: %.4f"%(accC))